This file contains code that ranks scenes based on the riskiness of the trajecotiry of SDC vehicle in the scenes.
The riskiness is measured in terms of minimum distance (over the course of the episode) of the SDC vehicle from a 
    vehicle with a high score on a certain heuristic.

In [1]:
# Set working directory to the base directory 'gpudrive'
import os
import sys
from pathlib import Path

working_dir = Path.cwd()
while working_dir.name != 'gpudrive-CoDec':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)
sys.path.append(str(working_dir))

In [2]:
from examples.CoDec_Research.code.construals.heuristic_functions import *

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

2025-06-25 15:20:42.472692: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750879242.487347   75073 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750879242.492709   75073 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750879242.508223   75073 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750879242.508255   75073 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750879242.508257   75073 computation_placer.cc:177] computation placer alr

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

In [3]:
training_config = load_config("examples/experimental/config/reliable_agents_params")
# dataset_path = "data/processed/gitIgnore_data/training/"
dataset_path = "data/processed/construal/Set1V2/"
intermediate_results_path = "examples/CoDec_Research/results/simulation_results/temp/"

moving_veh_masks = get_mov_veh_masks(
                                    training_config=training_config, 
                                    device="cpu", 
                                    dataset_path=dataset_path,
                                    max_agents=64,
                                    result_file_loc=intermediate_results_path,
                                    processID="HeuristicFilter",
                                    save_data = False,
                                    )
moving_veh_masks = {env_path2name(k): v for k, v in moving_veh_masks.items()}

training_config['max_controlled_agents'] = 1
env_config, train_loader, env, sim_agent = get_gpuDrive_vars(
                                                            training_config=training_config,
                                                            device="cpu",
                                                            num_parallel_envs=10,
                                                            dataset_path=dataset_path,
                                                            total_envs=10,
                                                            sim_agent_path="daphne-cornelisse/policy_S10_000_02_27",
                                                            )

Using moving vehicle masks from file: examples/CoDec_Research/results/simulation_results/temp/Set1V2_movVeh_masks_2025-05-14 19_21_06.319526.pickle
Initialized default environment


In [4]:
sublist_to_tuples = lambda x: [tuple(sublist) for sublist in x]
construal_indcs = {k: sublist_to_tuples(torch.nonzero(v).tolist()) for k, v in moving_veh_masks.items()}

In [5]:
heading_dev_dict = {}

In [6]:
# |Loop through all batches
for batch in tqdm(train_loader, desc=f"Processing Waymo batches",
                    total=len(train_loader), colour="blue"):
    # |BATCHING LOGIC: https://github.com/Emerge-Lab/gpudrive/blob/bd618895acde90d8c7d880b32d87942efe42e21d/examples/experimental/eval_utils.py#L316
    # |Update simulator with the new batch of data
    env.swap_data_batch(batch)
    heading_dev_dict.update(get_construal_dev_ego_heading(env, construal_indcs))

Processing Waymo batches: 100%|██████████| 1/1 [00:00<00:00,  2.07it/s]


In [7]:
heading_dev_dict

{'tfrecord-00034-of-00150_102': {(0,): np.float64(-0.008847544292945726),
  (1,): np.float64(-3.132717311503218),
  (2,): np.float64(-0.04593622368180372),
  (3,): np.float64(-0.019223669247831216),
  (4,): np.float64(-2.9934107086132116),
  (5,): np.float64(-0.42589082879409246),
  (6,): np.float64(-0.0018299717639873227),
  (7,): np.float64(-3.1401938936495717),
  (9,): np.float64(-0.02230424312786441),
  (10,): np.float64(-2.9355425412281013),
  (11,): np.float64(-3.0903024672585735),
  (12,): np.float64(-2.616071477859709),
  (13,): np.float64(-0.9408046125406775),
  (14,): np.float64(-0.31506320427687856),
  (15,): np.float64(-0.17599298360279972),
  (16,): np.float64(-0.21365146134541868)},
 'tfrecord-00034-of-00150_154': {(0,): np.float64(-1.5184234346167798),
  (1,): np.float64(-3.2532758321731734),
  (2,): np.float64(-3.542040372723984),
  (5,): np.float64(-0.19827717054259075),
  (6,): np.float64(-2.825758644971167),
  (7,): np.float64(-3.046814460765507)},
 'tfrecord-00070-o

In [13]:
moving_veh_masks.keys()

dict_keys(['tfrecord-00034-of-00150_102', 'tfrecord-00034-of-00150_154', 'tfrecord-00070-of-00150_182', 'tfrecord-00124-of-00150_62', 'tfrecord-00133-of-00150_121', 'tfrecord-00136-of-00150_130', 'tfrecord-00136-of-00150_6', 'tfrecord-00137-of-00150_154', 'tfrecord-00142-of-00150_36', 'tfrecord-00145-of-00150_75'])

In [14]:
env.data_batch

['data/processed/construal/Set1V2/tfrecord-00034-of-00150_102.json',
 'data/processed/construal/Set1V2/tfrecord-00034-of-00150_154.json',
 'data/processed/construal/Set1V2/tfrecord-00070-of-00150_182.json',
 'data/processed/construal/Set1V2/tfrecord-00124-of-00150_62.json',
 'data/processed/construal/Set1V2/tfrecord-00133-of-00150_121.json',
 'data/processed/construal/Set1V2/tfrecord-00136-of-00150_130.json',
 'data/processed/construal/Set1V2/tfrecord-00136-of-00150_6.json',
 'data/processed/construal/Set1V2/tfrecord-00137-of-00150_154.json',
 'data/processed/construal/Set1V2/tfrecord-00142-of-00150_36.json',
 'data/processed/construal/Set1V2/tfrecord-00145-of-00150_75.json']